In [1]:
import tensorflow as tf
import cv2
import numpy as np
from PIL import Image
from flask import request, jsonify, Flask
from keras.models import Sequential, load_model
from keras import backend as k
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import requests

In [2]:
# Create Flask app
app = Flask(__name__
            )

In [3]:
# Function to load the models
def load_model_fn(path):
    model = load_model(path)
    print('Model loaded!')
    return model

load = load_model_fn('./model.h5') 

c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
c:\Users\DELL\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\optimizers\base_optimizer.py:34: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model loaded!


In [4]:
classes = [
    "Adam Wahdan",
    "Anahita Hashemzadeh",
    "Aysel Hazem",
    "Aysel Mohamed",
    "Cailey Fleming",
    "Chandler Riggs",
    "Christina Ricci",
    "Emma Watson",
    "Finn Wolfhard",
    "Hala Al Turk",
    "haley joel osment",
    "Isaac Hempstead-Wright",
    "Jan Ramez",
    "Jana Amr",
    "Lareen Eloraby",
    "Lina Ayman",
    "Lolly Ali",
    "Macaulay Culkin",
    "Malak Mostafa",
    "Maria Gaafar",
    "Menna Arafa",
    "Millie Bobby Brown",
    "Mona Zaher",
    "Nelly Elbatran",
    "Polycarpus",
    "Reem Abdelkader",
    "Selim Mostafa",
    "Thomas Brodie-Stangster",
    "Tia Hamdy",
    "Younis Koura"
]

In [5]:
images = {
    "Adam Wahdan":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708722623/citizen/65d906f6c539b632795ad78d/image/znrmra5br9getnvsag3m.jpg",
    "Anahita Hashemzadeh":"https://res.cloudinary.com/dsxnevf3i/image/upload/v1710686061/citizen/image/dy3h5z3ekbznzckrpqjy.png",
    "Aysel Hazem":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708722207/citizen/65d906f6c539b632795ad78d/image/ytmlekgqesh2gmqk0f0k.jpg",
    "Aysel Mohamed":"https://res.cloudinary.com/dsxnevf3i/image/upload/v1711112583/citizen/image/pi05m2juezhjn8jhlxci.png",
    "Cailey Fleming":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708723002/citizen/65d906f6c539b632795ad78d/image/ge7o8msqejm8iayqteda.jpg",
    "Chandler Riggs":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708723198/citizen/65d906f6c539b632795ad78d/image/b6e9irhz72smmb84frhe.jpg",
    "Christina Ricci":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708723375/citizen/65d906f6c539b632795ad78d/image/qe4yc5qbell1flgaeccz.jpg",
    "Emma Watson":"https://res.cloudinary.com/dy8vniflb/image/upload/v1711276558/foundReport/65e502ec65ce123a3d64233c/image/yd3slk6aghlwilyoyino.webp",
    "Finn Wolfhard":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708723735/citizen/65d906f6c539b632795ad78d/image/tqpzo701dff7bmflhssr.jpg",
    "Hala Al Turk":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708723910/citizen/65d906f6c539b632795ad78d/image/hcl7tvehoveog2ffeia1.jpg",
    "haley joel osment":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708724065/citizen/65d906f6c539b632795ad78d/image/ohwkmuqox6ynndrw7oxr.jpg",
    "Isaac Hempstead-Wright":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708817454/citizen/65da5df32bca04136066bf93/image/pjzp3qn0foyyodwwevo5.jpg",
    "Jan Ramez":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708813968/citizen/65da5df32bca04136066bf93/image/vm52jyilj1letnhflwhu.jpg",
    "Jana Amr":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708814802/citizen/65da5df32bca04136066bf93/image/opz5txujfj0zx6qfludw.jpg",
    "Lareen Eloraby":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708815336/citizen/65da5df32bca04136066bf93/image/ogs7ypenidg4yifev4l5.jpg",
    "Lina Ayman":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708815819/citizen/65da5df32bca04136066bf93/image/i9roydliqwcjfvjswlxm.jpg",
    "Lolly Ali":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708811753/citizen/65da5df32bca04136066bf93/image/mouiswejrzpoel3cq4si.jpg",
    "Macaulay Culkin":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708816683/citizen/65da5df32bca04136066bf93/image/rw3tcnvted5kbst8g5kp.jpg",
    "Malak Mostafa":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708813464/citizen/65da5df32bca04136066bf93/image/yapi6vezwnv3p4chnpo7.jpg",
    "Maria Gaafar":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708814252/citizen/65da5df32bca04136066bf93/image/hbu9igweaulk1frn1jtr.jpg",
    "Menna Arafa":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708727698/citizen/65d906f6c539b632795ad78d/image/czumh3wbvlaou3r6hlkv.jpg",
    "Millie Bobby Brown":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708729032/citizen/65d906f6c539b632795ad78d/image/ony40ywmflghv6adhuw2.jpg",
    "Mona Zaher":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708729332/citizen/65d906f6c539b632795ad78d/image/yoikxzytycyji3hl3bbx.png",
    "Nelly Elbatran":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708738185/citizen/65d906f6c539b632795ad78d/image/f2wgcrmzk9tr3ssg2ejd.png",
    "Polycarpus":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708738456/citizen/65d906f6c539b632795ad78d/image/txebko7jbw5bpyiokfos.png",
    "Reem Abdelkader":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708738767/citizen/65d906f6c539b632795ad78d/image/srghg8lxays4c3ucmo3l.pngvalue",    
    "Selim Mostafa":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708739138/citizen/65d906f6c539b632795ad78d/image/d4zkp4hwqrjgsygvghsd.png",
    "Thomas Brodie-Stangster":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708739434/citizen/65d906f6c539b632795ad78d/image/deqq6ck3m3sq1ogfierm.jpg",
    "Tia Hamdy":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708739842/citizen/65d906f6c539b632795ad78d/image/ka9i4q8av6uo9jhtjkvo.jpg",
    "Younis Koura":"https://res.cloudinary.com/dy8vniflb/image/upload/v1708740140/citizen/65d906f6c539b632795ad78d/image/bxzyv8dmb2se5eud19pv.png"
}

In [6]:
# Function to preprocess the image
def preprocess_image(image_path, target_size):
    try:
        # Load the image
        image = Image.open(requests.get(image_path, stream=True).raw)
        # Convert the image to RGB
        rgb_image = image.convert("RGB")

        # Resize the RGB image
        resized_image = rgb_image.resize(target_size)

        # Convert the RGB image to a NumPy array
        image_array = np.array(resized_image)

        # Normalize the image by dividing pixel values by 255
        mean_value = 0.5
        std_value = 0.5
        image_array = (image_array - mean_value) / std_value

        # Add batch dimension
        image_array = np.expand_dims(image_array, axis=0)

        return image_array
    except Exception as e:
        print(f"Error preprocessing image: {str(e)}")
        return None

In [7]:
from flask_cors import CORS
# Create Flask app
app = Flask(__name__)
CORS(app)  # Enable CORS for all origins

In [8]:
@app.route('/predictApi', methods=['POST'])
def api():
    try:
        # Get the JSON data from the request
        data = request.json

        # Extract the image URL from the JSON data
        imageUrl = data.get('imageUrl')

        if not imageUrl:
            return jsonify({"error": "Image URL not provided."})

        # Preprocess the image using the provided URL
        image_arr = preprocess_image(imageUrl, target_size=(224, 224))

        # Handle error if image preprocessing fails
        if image_arr is None:
            return jsonify({"error": "Error processing image."})

        # Make predictions using the loaded model
        prediction = load.predict(image_arr)

        # Get the predicted class label index and confidence
        index_largest_value = np.argmax(prediction[0])
        confidence = prediction[0][index_largest_value]

        # Check if the confidence is below the threshold
        threshold = 0.7  # Set your desired confidence threshold here
        if confidence < threshold:
            return jsonify({"error": "Confidence below threshold."})

        result = classes[index_largest_value]

        # Check if the predicted class label is in the classes list
        if result not in classes:
            return jsonify({"error": "Photo not found."})

        # Return the prediction and confidence as JSON response
        return jsonify({"prediction": result, "confidence": float(confidence), "image": images[result]})
    except Exception as e:
        # Return error message as JSON response
        return jsonify({"error": str(e)})    

In [9]:
if __name__ == "__main__":
    # Run the Flask app
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 597ms/step


INFO:werkzeug:127.0.0.1 - - [05/May/2024 00:37:50] "POST /predictApi HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


...

INFO:werkzeug:127.0.0.1 - - [05/May/2024 00:40:37] "POST /predictApi HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


INFO:werkzeug:127.0.0.1 - - [05/May/2024 00:40:47] "POST /predictApi HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step


INFO:werkzeug:127.0.0.1 - - [05/May/2024 00:48:42] "POST /predictApi HTTP/1.1" 200 -
